In [150]:
from yolo_predict import YOLO_Pred
import cv2
import yaml
from yaml.loader import SafeLoader
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [4]:
print(cv2.__version__)

4.6.0


In [151]:
yolo = YOLO_Pred('/home/cremerf/FinalProject/detect/best.onnx', 'data.yaml')

In [152]:
df_predictions = yolo.predictions('/home/cremerf/FinalProject/data/images/test/test_7.jpg')

In [153]:
df_predictions

,Center_X,Center_Y,Width,Height,Confidence,ProbabilityScore
572,98.517586,58.344646,19.973438,24.347050,0.001154,0.999977
1059,155.107376,107.543571,17.532705,13.032552,0.000142,0.999990
581,174.102814,61.129448,22.131855,16.180351,0.000332,0.999981
585,204.683701,56.656437,18.419516,10.686930,0.000040,0.999977
1203,24.865353,126.568062,22.158545,29.920956,0.008113,0.999988
...,...,...,...,...,...,...
1145,203.527283,115.974922,23.511864,17.438477,0.002097,0.999991
712,573.866516,65.199333,29.849636,8.114309,0.000128,0.999991
310,557.077026,26.599327,32.811779,24.781599,0.000061,0.999990
600,323.434723,56.862175,22.192471,12.150562,0.000072,0.999977


In [93]:
img=cv2.imread('/home/cremerf/FinalProject/data/images/test/test_7.jpg')

# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [94]:
# load YAML
with open('/home/cremerf/FinalProject/detect/data.yaml', mode='r') as f:
    data_yaml = yaml.load(f,Loader=SafeLoader)
    
labels = data_yaml['names']
print(labels)

['object']


In [95]:
# load YOLO model
yolo = cv2.dnn.readNetFromONNX('/home/cremerf/FinalProject/detect/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# load the image
img = cv2.imread('/home/cremerf/FinalProject/data/images/test/test_7.jpg')
image = img.copy()
row, col, d = image.shape

# get the YOLO prediction from the the image
# step-1 convert image into square image (array)
max_rc = max(row,col)
input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
input_image[0:row,0:col] = image

# step-2: get prediction from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
yolo.setInput(blob)
preds = yolo.forward() # detection or prediction from YOLO

In [96]:
labels = pd.DataFrame(preds[0])
labels.columns = ['Center_X','Center_Y','Width','Height', 'Confidence','ProbabilityScore']

In [139]:
# Non Maximum Supression
# step-1: filter detection based on confidence (0.4) and probability score (0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

# widht and height of the image (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] # confidence of detection an object
    if confidence > 0.4:
        class_score = row[5:].max() # maximum probability from 20 objects
        class_id = row[5:].argmax() # get the index position at which max probabilty occur
        
        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            # construct bounding from four values
            # left, top, width and height
            left = int((cx - 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            
            box = np.array([left,top,width,height])
            
            # append values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)
            
# clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# NMS
index = np.array(cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45)).flatten()

In [ ]:
# Draw the Bounding
for ind in index:
    # extract bounding box
    x,y,w,h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]*100)
    
    
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),(255,255,255),-1)
    
    cv2.putText(image,str(ind),(x,y-10),cv2.FONT_HERSHEY_PLAIN,2,(0,0,0),1)

In [ ]:
cv2.imwrite(filename='test_neightbours.jpg',img=image)

#### 1er algoritmo/heuristica


    * Detecta bounding box vecino de la izquierda
        * TODO --> mismos calculos pero para la derecha y arriba


In [ ]:
df_predictions.head(5)

In [65]:
df_predictions.iloc[0][1]

58.344646

In [61]:
X_center_0 = df_predictions.iloc[0][0]
Y_center_0 = df_predictions.iloc[0][1]
Width_0 = df_predictions.iloc[0][2]
height_0 = df_predictions.iloc[0][3]
threshold_x_0 = 0.2 * Width_0
threshold_y_0 = 0.2 * height_0


In [ ]:
x_min = X_center_0 - Width_0 - (0.5*threshold_x_0)

In [ ]:
x_max = X_center_0 - Width_0 + (0.5*threshold_x_0)

In [ ]:
y_min = Y_center_0 - (0.5*threshold_x_0)

In [ ]:
y_max = Y_center_0 + (0.5*threshold_x_0)

In [ ]:
df_predictions.iloc[1][0]

In [146]:
df_predictions.loc[379]

Center_X            473.648102
Center_Y             33.051300
Width                23.014057
Height               11.927161
Confidence            0.000122
ProbabilityScore      0.999982
Name: 379, dtype: float32

In [145]:
df_predictions.loc[1126]

Center_X             50.824856
Center_Y            112.998108
Width                11.555897
Height               10.522651
Confidence            0.000614
ProbabilityScore      0.999992
Name: 1126, dtype: float32

In [ ]:
list_of_neightbours = []
list_of_alones = []
for index_bb in df_predictions.index:
    for index in df_predictions.index:
        X_center_0 = df_predictions.loc[index_bb][0]
        Y_center_0 = df_predictions.loc[index_bb][1]
        Width_0 = df_predictions.loc[index_bb][2]
        height_0 = df_predictions.loc[index_bb][3]
        threshold_x_0 = 0.5 * Width_0
        threshold_y_0 = 0.5 * height_0

        X_center_neightbour = df_predictions.loc[index][0]
        Y_center_neightbour = df_predictions.loc[index][1]


        x_min = X_center_0 - Width_0 - (0.5*threshold_x_0)
        x_max = X_center_0 - Width_0 + (0.5*threshold_x_0)


        y_min = Y_center_0 - (0.5*threshold_y_0)

        y_max = Y_center_0 + (0.5*threshold_y_0)
    
        if x_min < X_center_neightbour < x_max and y_min < Y_center_neightbour < y_max:
            print(f'El bounding box {index_bb} tiene al bb {index} de vecino')
            list_of_neightbours.append(index_bb)
        else:
            print(f'El bounding box {index_bb} no tiene vecinos')
            list_of_alones.append(index_bb)

    

In [135]:
len(list_of_neightbours)

55

In [140]:
list_of_neightbours

[585,
 576,
 577,
 583,
 574,
 580,
 1297,
 496,
 1235,
 579,
 579,
 430,
 230,
 1174,
 66,
 587,
 273,
 143,
 143,
 422,
 225,
 277,
 185,
 1318,
 397,
 187,
 317,
 1176,
 303,
 303,
 304,
 149,
 177,
 1055,
 229,
 299,
 227,
 227,
 227,
 1152,
 234,
 234,
 92,
 461,
 838,
 616,
 298,
 298,
 631,
 330,
 632,
 59,
 1320,
 836,
 23]

In [132]:
len(list_of_alones)

25866

In [ ]:
yolo.predct

In [ ]:
df_predictions.loc[list_of_neightbours]